# 1.2 Reading a 2D vector field from a simulation

In [1]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

/home/dharrington/cyclops/src


First we define the objects we will use - the mesh reader (to read the mesh from the FEM file), the pickle manager (to save the mesh data that we want to analyse), and the unfolder (to manage the mesh).

In [2]:
from sim_reader import MeshReader, Unfolder
from fields import ScalarField, VectorField
from object_reader import PickleManager
from regressors import LModel, CSModel
import numpy as np


# Define the necessary objects
reader = MeshReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

<meshio mesh object>
  Number of points: 87200
  Number of cells:
    hexahedron27: 2016
    hexahedron27: 3360
    hexahedron27: 4704
  Point sets: right, top, left, , centre_x_bottom_y_back_z, centre_x_bottom_y_front_z, left_x_bottom_y_centre_z, right_x_bottom_y_centre_z
  Point data: disp_x, disp_y, disp_z, temperature
  Cell data: vonmises_stress


Next we read the simulation data. Change the sensor_region from `right` to whatever you called the region where sensors can be placed in your FEM file.

We get an array of 3D positions of all the nodes in that region in the mesh. In the monoblock file they all have x=0, but in your file they could all have z=5 or y=3 or whatever. The `compress_2D` function removes the x components so that the array becomes an array of 2D positions. You will have to redefine this function if you are analysing a different face or surface.

The way the mesh reader works is if you have a vector `disp` then it splits it into 3 scalars `disp_x`, `disp_y`, and `disp_z`. You can find out the names of your vector by looking at the mesh info that is output by the code above (if you use a vector called something other than `disp`). We read the components of the vector and then join them together into a `VectorField`.

We then save this field.

In [3]:
# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
grid = unfolder.generate_grid(bounds, 30, 30)

disp = np.array([
    reader.read_scalar(sensor_region, 'disp_x'),
    reader.read_scalar(sensor_region, 'disp_y'),
    reader.read_scalar(sensor_region, 'disp_z')
]).T

disp_field = VectorField(LModel, bounds)
disp_field.fit_model(pos_2D, disp)

# Save the simulation data
pickle_manager.save_file('simulation', 'disp_plane_field.obj', disp_field)
pickle_manager.save_file('simulation', 'disp_plane_points.obj', grid)

The displacement field is **NOT** uniform in the horizontal direction, but it doesn't change much and we can pretend that it is uniform if we want to get a 1D vector field. The code below considers the 1D strip along the edge of the field.

In [4]:
# Convert the 2D field to a 1D line
pos1 = (bounds[0][0], bounds[0][1])
pos2 = (bounds[0][0], bounds[1][1])
line_2D = unfolder.generate_line(pos1, pos2, 50)
line_disps = disp_field.predict_values(line_2D)
line_1D = unfolder.compress_1D(line_2D)

bounds_1D = np.array([line_1D[0], line_1D[-1]])
new_line_field = VectorField(CSModel, bounds_1D)
new_line_field.fit_model(line_1D, line_disps)

# Save the new 1D line field
pickle_manager.save_file('simulation', 'disp_line_field.obj', new_line_field)
pickle_manager.save_file('simulation', 'disp_line_points.obj', line_1D)